<a href="https://colab.research.google.com/github/XieSandi/DicodingMLCourse-Exercise/blob/main/rock_paper_scissors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Load Dataset**

In [ ]:
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

--2021-04-12 19:53:09--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322873683 (308M) [application/zip]
Saving to: ‘/tmp/rockpaperscissors.zip’

/tmp/rockpapersciss 100%[===================>] 307.92M   724KB/s    in 4m 13s  

2021-04-12 19:57:23 (1.22 MB/s) - ‘/tmp/rockpaperscissors.zip’ saved [322873683/322873683]



**Installing Library / Package**

In [ ]:
pip install split-folders

**Importing Library**

In [ ]:
import tensorflow as tf
import splitfolders
import zipfile
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator

**Exctracting dataset**

In [ ]:
local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
os.listdir('/tmp/rockpaperscissors')

['paper', 'scissors', 'rock', 'README_rpc-cv-images.txt', 'rps-cv-images']

**Spliting**

In [ ]:
main_dir = "/tmp/rockpaperscissors/rps-cv-images"
splitfolders.ratio(main_dir , output="/tmp/rockpaperscissors", seed=1337, ratio=(.6, .4), group_prefix=None)

train_dir = os.path.join('/tmp/rockpaperscissors/', 'train')
validate_dir = os.path.join('/tmp/rockpaperscissors/', 'val')

Copying files: 2188 files [00:00, 4014.08 files/s]


In [ ]:
# training
train_paper = os.path.join(train_dir, 'paper')
train_rock = os.path.join(train_dir, 'rock')
train_scissor = os.path.join(train_dir, 'scissor')
 
#validating
validation_paper = os.path.join(validate_dir, 'paper')
validation_rock = os.path.join(validate_dir, 'rock')
validation_scissor = os.path.join(validate_dir, 'scissor')

In [ ]:
 train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')
 
test_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')
 
validation_generator = test_datagen.flow_from_directory(
        validate_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

Found 1312 images belonging to 3 classes.
Found 876 images belonging to 3 classes.


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='sigmoid')
])

In [ ]:
# compile model with 'adam' optimizer loss function 'binary_crossentropy' 
model.compile(loss='binary_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

# latih model with model.fit 
model.fit(
      train_generator,
      steps_per_epoch=25,
      epochs=20,
      validation_data=validation_generator,
      validation_steps=5,
      verbose=2)

Epoch 1/20
25/25 - 43s - loss: 0.1485 - accuracy: 0.9250 - val_loss: 0.1532 - val_accuracy: 0.9250
Epoch 2/20
25/25 - 42s - loss: 0.1319 - accuracy: 0.9200 - val_loss: 0.1922 - val_accuracy: 0.9000
Epoch 3/20
25/25 - 42s - loss: 0.1176 - accuracy: 0.9525 - val_loss: 0.1395 - val_accuracy: 0.9250
Epoch 4/20
25/25 - 42s - loss: 0.0737 - accuracy: 0.9500 - val_loss: 0.1281 - val_accuracy: 0.9625
Epoch 5/20
25/25 - 42s - loss: 0.0776 - accuracy: 0.9638 - val_loss: 0.0594 - val_accuracy: 0.9688
Epoch 6/20
25/25 - 42s - loss: 0.0642 - accuracy: 0.9638 - val_loss: 0.1893 - val_accuracy: 0.9312
Epoch 7/20
25/25 - 42s - loss: 0.0623 - accuracy: 0.9613 - val_loss: 0.0938 - val_accuracy: 0.9563
Epoch 8/20
25/25 - 42s - loss: 0.0502 - accuracy: 0.9775 - val_loss: 0.1337 - val_accuracy: 0.9500
Epoch 9/20
25/25 - 42s - loss: 0.0494 - accuracy: 0.9775 - val_loss: 0.0843 - val_accuracy: 0.9688
Epoch 10/20
25/25 - 42s - loss: 0.0625 - accuracy: 0.9675 - val_loss: 0.1187 - val_accuracy: 0.9375
Epoch 11/

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

uploaded = files.upload()
 
for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150, 150))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  
  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  
  #String output template
  paper     = "[[1. 0. 0.]]"
  rock      = "[[0. 1. 0.]]"
  scissors  = "[[0. 0. 1.]]"
  
  #convert numpy array into string
  result = np.array_str(classes) 
  
  
  print('\n')
  print('Predict RESULT')
  print(fn)
  print(result)


  #Changing result into human language
  if result == paper :
     print("paper")
  elif result == rock :
     print("rock")
  elif result == scissors :
     print("scissors")
  else :
     print("[[%PAPER. %ROCK. %SCISSORS]]")